In [ ]:
import csv

csv_file_path = 'news_stock_price.csv'

with open(csv_file_path, newline='') as csvfile:
    myreader = csv.reader(csvfile)

    next(myreader)
    headers = next(myreader)
    print("Column names:", headers)

    print(headers[1])
    row_count = 0
    for row in myreader:
        if row_count == 50:
            break
        if row:
            for column in row:
                print(column)

            print()
        row_count += 1


In [110]:
#TOKENIZING AND NORMALIZNG THE FULL TEXT COLUMN
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import bigrams
from collections import Counter

csv_file_path = 'news_stock_price.csv'

with open(csv_file_path, newline='') as csvfile:
    myreader = csv.reader(csvfile)
    next(myreader)
    
    ft_rows = []
    for row in myreader:
        ft_rows.append(row[2])

    row_count = 0
    for text in ft_rows:
        if row_count < 50:
            row_count += 1
            
    def custom_tokenize(text):
        protected_text = text.replace('S&P 500', 'SNP500')
        tokens = nltk.word_tokenize(protected_text)
        restored_tokens = []
        for token in tokens:
            if token == 'SNP500':
                restored_tokens.extend(['S&P', '500'])
            else:
                restored_tokens.append(token)
        
        return restored_tokens

    combined_text = ' '.join(ft_rows)

    stop_words = stopwords.words('english')
    stop_words.extend(["said", "also", "like"])
    
    tokens = custom_tokenize(combined_text)
    
    normalized_tokens = [word for word in tokens if (word.isalpha() and word.lower() not in stop_words)or word in ['S&P', '500']]

In [ ]:
#FINDING OUT STATISTICS ABOUT OUR DATA
import csv
from nltk.tokenize import sent_tokenize

total_sentences = 0
total_words = 0
total_tokens = 0
unique_tokens = set()
unique_words = set()

for text in ft_rows:
    sentences = sent_tokenize(text)
    total_sentences += len(sentences)

    report_tokens = custom_tokenize(text)
    total_tokens += len(report_tokens)
    total_words += len([token for token in report_tokens if token.isalpha()])

    unique_tokens.update(report_tokens)
    unique_words.update([token.lower() for token in report_tokens if token.isalpha()])

average_sentences = total_sentences / len(ft_rows) if ft_rows else 0
average_words = total_words / len(ft_rows) if ft_rows else 0
average_tokens = total_tokens / len(ft_rows) if ft_rows else 0

num_unique_tokens = len(unique_tokens)
num_unique_words = len(unique_words)

print("Average number of sentences per report:", average_sentences)
print("Average number of words per report:", average_words)
print("Average number of tokens per report:", average_tokens)
print("Total number of unique words:", num_unique_words)
print("Total number of unique tokens:", num_unique_tokens)


In [ ]:
#PRINTS OUT THE FIRST 500 tokens in the tokenized and normalized list
tcount = 0
tstring = ''
for token in normalized_tokens:
    if tcount < 500:
        tstring += token + ' '
        tcount += 1
print(tstring)

In [ ]:
#PRINTS THE TOP 50 WORDS OF NEW TOKENIZED AND NORMALIZED LIST
word_freq = nltk.FreqDist(normalized_tokens)
top_50_words = word_freq.most_common(50)

for word, freq in top_50_words:
    print(f"{word}: {freq}")


In [ ]:
from nltk.util import bigrams

bigram_list = list(bigrams(normalized_tokens))

bigram_counts = Counter(bigram_list)
top_10_bigrams = bigram_counts.most_common(10)

print("Top 10 Bigrams:")
for bigram, count in top_10_bigrams:
    print(f"{bigram}: {count}")

In [ ]:
#MAKING A WORD CLOUD FOR ALL TOKENS IN FULL_TEXT
#IM NOT SURE IF WE SHOULD LEMMATIZE OR NOT, EITHER WAY S&P doesnt show up in cloud
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

all_lemmas = [lemmatizer.lemmatize(w) for w in normalized_tokens]

text = " ".join(all_lemmas)

from wordcloud import WordCloud

word_freq = nltk.FreqDist(text)

top_words = dict(word_freq.most_common(100))
wordcloud = WordCloud(background_color='white').generate_from_frequencies(top_words)


import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")
plt.show()


In [99]:
#FILTERS THE DATASET SO ONLY FULL TEXT REPORTS 
#WHERE A SPECIFIC WORD IS MENTIONED IN IT ARE INCLUDED. IT THEN ADDS THOSE REPORTS
#TO A NEW LIST(apple_rows) OF REPORTS THAT ALL INCLUDE THAT WORD 
#WE CAN USE THIS TO FIND BIGRAMS, DISTRIBUTIONS, and WORD CLOUDS FOR ALL REPORTS
#THAT MENTION THAT COMPANY

import csv
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import bigrams
from collections import Counter
import string
import spacy

csv_file_path = 'news_stock_price.csv'

column_name_to_search = 'full_text'

#SPECIFIC WORD TO MENTION. We can use this to find reports that mention a specific company
word_to_search = 'Tesla'

apple_rows = []

with open(csv_file_path, newline='') as csvfile:
    myreader = csv.reader(csvfile)

    next(myreader)
    headers = next(myreader)

    column_index = None

    column_index = headers.index(column_name_to_search)
    
    if column_index is not None:
        for row in myreader:
            if len(row) > column_index:
                if word_to_search in row[column_index]:
                    apple_rows.append(row[2])
apple_count = 0
for text in apple_rows:
    apple_count += text.count(word_to_search)
print(apple_count)


1147


In [ ]:
#TOKENIZES AND NORMALIZES THE TEXT THAT MENTIONS THE COMPANY IN THE CELL ABOVE
#PRETTY MUCH THE SAME CODE AS I TOKENIZED BEFORE
#WE CAN ADD MORE WORDS/PHRASES TO CUSTOM TOKENIZE IF WE WANT TO PROTECT THEM (MAYBE % SIGN)
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import bigrams
from collections import Counter

def custom_tokenize(text):
    protected_text = text.replace('S&P 500', 'SNP500')
    tokens = nltk.word_tokenize(protected_text)

    restored_tokens = []
    for token in tokens:
        if token == 'SNP500':
            restored_tokens.extend(['S&P', '500'])
        else:
            restored_tokens.append(token)
    
    return restored_tokens

combined_text = ' '.join(apple_rows)

stop_words = stopwords.words('english')
stop_words.extend(["said", "also", "like"])

tokens = custom_tokenize(combined_text)
normalized_tokens = [word for word in tokens if (word.isalpha() and word.lower() not in stop_words) or word in ['S&P', '500']]
bigram_list = list(bigrams(normalized_tokens))

bigram_counts = Counter(bigram_list)
top_10_bigrams = bigram_counts.most_common(10)

print("Top 10 Bigrams:")
for bigram, count in top_10_bigrams:
    print(f"{bigram}: {count}")


In [ ]:
#FINDING COLLOCATIONS
from nltk.collocations import *

def find_collocations(tokens, num):
    bigram_measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(normalized_tokens)

    finder.apply_freq_filter(10)
    top_n_bigrams = finder.nbest(bigram_measures.pmi, num)
    bigram_freqs = [(bigram, finder.ngram_fd[bigram]) for bigram in top_n_bigrams]
    return bigram_freqs
apple_collocations = find_collocations(normalized_tokens, 10)

print("Top 10 Collocations:")
for bigram, count in apple_collocations:
    print(f"{bigram}: {count}")

In [ ]:
#PRINTS FIRST 500 TOKENS AS STRING
newstring = ''
tcount = 0
for token in normalized_tokens:
    if tcount < 10000:
        newstring += token + ' '
        tcount +=1


In [ ]:
import csv
from collections import defaultdict, Counter
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

target_words = ["Apple", "Facebook", "Microsoft", "Tesla", "Amazon"]
word_frequency_over_time = defaultdict(Counter)

with open('news_stock_price.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        news_text = row[2]
        date = datetime.strptime(row[0], '%Y-%m-%d')

        tokens = news_text.split()
        word_counts = Counter(word for word in tokens if word in target_words)
        
        word_frequency_over_time[date].update(word_counts)

df = pd.DataFrame.from_dict(word_frequency_over_time, orient='index')
df = df.fillna(0) 

df.plot(figsize=(12, 6))
plt.xlabel('Date')
plt.ylabel('Word Frequency')
plt.title('Word Frequency Trends Over Time')
plt.legend(target_words) 
plt.xticks(rotation=45)
plt.grid(True)

plt.savefig('word_frequency_trends.png', dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
import spacy
import csv
from collections import Counter

nlp = spacy.load("en_core_web_lg")

def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ in ["ORG", "GPE", "PRODUCT"]]
entity_counter = Counter()

with open('news_stock_price.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        news_text = row[2]
        entities = extract_entities(news_text)
        entity_counter.update(entities)

print("Most common entities:")
for entity, count in entity_counter.most_common(10):
    print(f"{entity}: {count}")

In [ ]:
import matplotlib.pyplot as plt


entities = ["U.S.", "Fed", "Nasdaq","Russia", "NYSE", "Ukraine", "China", "Treasury"]
frequencies = [5556 + 2520, 5403, 1495+1221, 2126, 2022, 1877, 1664, 1234]

plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.bar(entities, frequencies, color='blue')
plt.xlabel('Entities')
plt.ylabel('Frequency')
plt.title('Most Common Entities Histogram (Combined U.S. and US)')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()
